In [83]:
import pandas as pd
import numpy as np
import configparser

In [84]:
dataset = pd.read_csv('../data/raw/train.csv')
dataset.head()

,CustomerID,Age,Gender,Income,VisitFrequency,AverageSpend,PreferredCuisine,TimeOfVisit,GroupSize,DiningOccasion,MealType,OnlineReservation,DeliveryOrder,LoyaltyProgramMember,WaitTime,ServiceRating,FoodRating,AmbianceRating,HighSatisfaction
0,1457,34,Female,138842,Weekly,162.954929,Indian,Dinner,8,Celebration,Dine-in,1,0,1,4.228618,1,5,3,1
1,1371,54,Male,98671,Rarely,66.918873,Indian,Lunch,4,Casual,Takeaway,0,0,0,50.247186,1,2,2,0
2,1505,47,Male,122351,Monthly,94.127670,Indian,Lunch,7,Casual,Takeaway,0,1,0,10.174873,3,3,5,0
3,1011,38,Male,78868,Weekly,92.705568,Mexican,Dinner,4,Business,Takeaway,0,0,1,14.237746,4,1,4,0
4,1016,50,Female,128686,Monthly,166.931144,American,Dinner,8,Business,Dine-in,0,0,0,56.319628,4,3,4,0


In [ ]:
dataset.dtypes

CustomerID                int64
Age                       int64
Gender                   object
Income                    int64
VisitFrequency           object
AverageSpend            float64
PreferredCuisine         object
TimeOfVisit              object
GroupSize                 int64
DiningOccasion           object
MealType                 object
OnlineReservation         int64
DeliveryOrder             int64
LoyaltyProgramMember      int64
WaitTime                float64
ServiceRating             int64
FoodRating                int64
AmbianceRating            int64
HighSatisfaction          int64
dtype: object

In [79]:
config = configparser.ConfigParser()
config.read('../pipeline.cfg')
valor = config.get('GENERAL', 'VARS_TO_DROP')
print(valor)

CustomerID,WaitTime,AverageSpend


In [63]:
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer

In [64]:
x_feature = dataset.drop(labels=list(config.get('GENERAL', 'VARS_TO_DROP').split(',')), axis=1)
y_target = dataset[config.get('GENERAL', 'TARGET')]
x_train, x_test, y_train, y_test = train_test_split(x_feature, y_target, test_size=0.3, shuffle=True, random_state=2025)

In [65]:
# Codificación de las variables categóricas fuera del pipeline
from sklearn.pipeline import Pipeline
from feature_engine.encoding import CountFrequencyEncoder
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [66]:
# Codificar las variables categóricas con LabelEncoder
label_encoding_vars = list(config.get('CATEGORICAL', 'LABEL_ENCODER').split(','))

def label_encode_columns(X, label_encoding_vars):
    X = X.copy()
    for col in label_encoding_vars:
        X[col] = LabelEncoder().fit_transform(X[col])
    return X

In [67]:
# Crear un pipeline donde la codificación se aplique externamente
high_satisfaction_model = Pipeline([
    ('categorical_encode_frequency', CountFrequencyEncoder(encoding_method='count', variables=label_encoding_vars)),
    ('feature_scaling', StandardScaler())
])

In [68]:
dataset.dtypes

CustomerID              category
Age                     category
Gender                    object
Income                  category
VisitFrequency            object
AverageSpend            category
PreferredCuisine          object
TimeOfVisit               object
GroupSize               category
DiningOccasion            object
MealType                  object
OnlineReservation       category
DeliveryOrder           category
LoyaltyProgramMember    category
WaitTime                category
ServiceRating           category
FoodRating              category
AmbianceRating          category
HighSatisfaction        category
dtype: object

-----------------------------

In [69]:
# Preprocesar los datos de entrenamiento antes de alimentar al pipeline
X_train_encoded = label_encode_columns(x_train, label_encoding_vars)
high_satisfaction_model.fit(X_train_encoded, y_train)

TypeError: Some of the variables are not categorical. Please cast them as object or categorical before using this transformer.